In [59]:
import pyshark
import os
import pandas
from netml.pparser.parser import PCAP
from netml.utils.tool import dump_data
import pprint

In [60]:
dataDir = "/code/2.Malwares/"
targetFile = "Zeus.pcap"
destination = dataDir + "out"
tsharkPath="/usr/bin/tshark"

In [61]:

parsedPcap = PCAP(dataDir + targetFile,
    flow_ptks_thres=2,
 )
parsedPcap.pcap2flows(q_interval=0.9)


'_pcap2flows()' starts at 2021-04-18 17:01:29
pcap_file: /code/2.Malwares/Zeus.pcap
ith_packets: 0
ith_packets: 10000
ith_packets: 20000
ith_packets: 30000
ith_packets: 40000
ith_packets: 50000
ith_packets: 60000
ith_packets: 70000
ith_packets: 80000
ith_packets: 90000
len(flows): 21741
total number of flows: 21741. Num of flows < 2 pkts: 4765, and >=2 pkts: 16976 without timeout splitting.
kept flows: 16585. Each of them has at least 2 pkts after timeout splitting.
flow_durations.shape: (16585, 1)
          col_0
count 16585.000
mean     11.030
std     506.590
min       0.000
25%       0.000
50%       1.365
75%       1.394
max   32587.277
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16585 entries, 0 to 16584
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col_0   16585 non-null  float64
dtypes: float64(1)
memory usage: 129.7 KB
None
0th_flow: len(pkts): 4
1000th_flow: len(pkts): 6
2000th_flow: len(pkts): 3
3000th_flow

In [69]:
parsedPcap.label_flows(label_file=None,label=0)
pprint.pprint(parsedPcap.flows[:5])
pprint.pprint(parsedPcap.labels[:5])





'_label_flows()' starts at 2021-04-18 17:09:09
'_label_flows()' ends at 2021-04-18 17:09:09 and takes 0.0003 mins.
[(('192.168.0.250', '239.255.255.250', 49152, 3702, 17),
  [<Ether  dst=01:00:5e:7f:ff:fa src=08:00:27:3c:8d:c9 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=1017 id=68 flags= frag=0 ttl=1 proto=udp chksum=0x414 src=192.168.0.250 dst=239.255.255.250 |<UDP  sport=49152 dport=3702 len=997 chksum=0xc3ad |<Raw  load='<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:wsa="http://schemas.xmlsoap.org/ws/2004/08/addressing" xmlns:wsd="http://schemas.xmlsoap.org/ws/2005/04/discovery" xmlns:wsdp="http://schemas.xmlsoap.org/ws/2006/02/devprof" xmlns:pub="http://schemas.microsoft.com/windows/pub/2005/07"><soap:Header><wsa:To>urn:schemas-xmlsoap-org:ws:2005:04:discovery</wsa:To><wsa:Action>http://schemas.xmlsoap.org/ws/2005/04/discovery/Hello</wsa:Action><wsa:MessageID>urn:uuid:fbbc82fb-9bdb-4631-a88a-32f048603c70</wsa:Message

In [63]:
parsedPcap.flow2features("IAT", fft=False, header=True)


'_flow2features()' starts at 2021-04-18 17:02:38
True
'_flow2features()' ends at 2021-04-18 17:02:44 and takes 0.1052 mins.


In [64]:

# dump data to disk
X, y = parsedPcap.features, parsedPcap.labels
outFilePath = destination+"/test.dat"
dump_data((X,y), out_file=outFilePath)

In [65]:
print(parsedPcap.features)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  6.29425049e-05 1.05984926e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.28000000e+02
  2.69412994e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.28000000e+02
  3.00407410e-05 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.28000000e+02
  1.90734863e-05 0.00000000e+00]
 [0.00000000e+00 6.00000000e+00 0.00000000e+00 ... 1.28000000e+02
  2.09808350e-05 6.75967932e-01]
 [0.00000000e+00 0.00000000e+00 3.00000000e+00 ... 4.20000000e+01
  6.76491022e-01 6.79682016e-01]]


In [66]:
print(len(parsedPcap.labels))


19254
